In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

## 1. Data Preprocessing

In [13]:
columns_to_remove=[]

### 1.1 Data Loading And Analysis

In [4]:
train_df=pd.read_csv('train.csv')
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### 1.2 Analyze and Replace / Remove Missing Data

In [11]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
train_df.isna().sum() * 100 / len(train_df)

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

Three columns are mising values. Most significant missing variables are the below:

* 1. Cabin: over 77% missing. is it even worth keeping given how many are missing.
* 2. Age: this is quite a significant missing dataset, given that age (combined with gender) were a big factor in whether you were evacuated first.
* 3. Embarked: less than 1%. Extrapolate.

Makes sense to just remove the cabin columns, so will add it to the list of columns_to_remove.

In [14]:
columns_to_remove.append('cabin')

### 1.3 Replace Missing Age Values

There are a number of ways to do this. 

* 1. Just apply a blanket average to the missing values. However, given that I intend to investigate the role that the "women and children" evacuation policy had, I will need to come up with a better approach.
* 2. Take averages based on a number of columns: use gender, titles, parch to narrow down the expected age of the missing values.

Will investigate the values of parch and sibsp to see if any trends can be determined.

In [23]:
parch = train_df.loc[(train_df['Parch'] > 0)]
parch

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q
24,25,0,3,"Palsson, Miss. Torborg Danira",female,8.0,3,1,349909,21.0750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
856,857,1,1,"Wick, Mrs. George Dennick (Mary Hitchcock)",female,45.0,1,1,36928,164.8667,NaN,S
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
869,870,1,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,NaN,S
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S


In [ ]:
sibsp = train_df.loc[(train_df['SibSp'] > 0)]
sibsp

In [24]:
parch['Age'].isna().sum()

16

In [25]:
none_parch = train_df.loc[((train_df['Parch'] == 0) & (train_df['SibSp'] == 0))]


In [26]:
none_parch['Age'].isna().sum()

133